# Introduction

Our business problem began by identifying the most common products bought together and for that we did several recommendation systems.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
try:
    df = pd.read_csv('orders.csv')
except:
    df = pd.read_csv('orders.xlsx')
df.drop(columns="Unnamed: 0", axis =1, inplace=True)
df.head()

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id
0,1128254,6346669,2023-01-06,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,600.0,Não,AMB,Bebidas,Aguas Minerais,1
1,1001096,6001131,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2
2,1001096,6001131,2023-01-17,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,3
3,1121833,6358142,2023-01-04,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,12.0,Não,AMB,Bebidas,Aguas Minerais,4
4,1122758,6328488,2023-01-04,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,60.0,Não,AMB,Bebidas,Aguas Minerais,5


How the company works is that for every deivery place in a day a truck takes the products, there can be more than one delivery place per day, as well several clients can have the same delivery place

In [2]:
#Implementing a orders Id
#order = df.groupby(['date','delivery_place']).size().reset_index()
#order['order_id'] = [i+1 for i in range(len(order))]
#order = order.drop(0,axis=1)
#data=df.merge(order, on=['date','delivery_place'])
#data.head()

In [3]:
#make groupby to get indexes product_tyoe and subtype
analise = df.groupby(['warehouse_zone','product_type','product_subtype'])['quantity'].sum().reset_index()
analise2 = df.groupby(['warehouse_zone','product_type',])['quantity'].sum().reset_index()

#Create Id's 
analise['cellule'] = (analise.groupby('product_type').cumcount()+1).apply(lambda x: str(x).zfill(2))
analise2['ID'] = (analise2.groupby('warehouse_zone').cumcount()+1).apply(lambda x: str(x).zfill(2))
analise2['alley']= analise2.apply(lambda row: row['warehouse_zone'] +'-'+ str(row['ID']), axis=1)

#Drop unecessary columns
analise2=analise2.drop(['quantity','ID'], axis=1)
analise=analise.drop(['quantity'],axis=1)

#Merge everything together on a dataframe
df_locations = df.merge(analise2, on=['warehouse_zone','product_type'], how='left')
df_new = df_locations.merge(analise, on=['warehouse_zone','product_type','product_subtype'], how='left')
df_new['alleycell'] = df_new.apply(lambda row: row['alley'] +'-'+ str(row['cellule']), axis=1)
df_new.head(3)

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id,alley,cellule,alleycell
0,1128254,6346669,2023-01-06,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,600.0,Não,AMB,Bebidas,Aguas Minerais,1,AMB-01,02,AMB-01-02
1,1001096,6001131,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2,AMB-01,02,AMB-01-02
2,1001096,6001131,2023-01-17,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,3,AMB-01,02,AMB-01-02


In [4]:
try:
    coords = pd.read_csv("coords.xls", delimiter=';')
except:
    coords = pd.read_csv("coords.csv", delimiter=';')
coords['x'] = coords['x'].str.replace(',', '.').astype(float)
coords['y'] = coords['y'].str.replace(',', '.').astype(float)

gg = df_new.groupby(['warehouse_zone','product_type','alley']).size().reset_index()
s= coords.merge(gg, on=['warehouse_zone','product_type'], how='left')
s= s.drop(0,axis=1)
s.loc[len(s)] = ['START',"START", 15, 1,"START"]
s.tail()

,warehouse_zone,product_type,x,y,alley
51,REF,Peixe,5.5,34.0,REF-09
52,REF,Refeições Cook & Chill (Socigeste),5.5,32.0,REF-10
53,SAL,Legumes,27.0,25.5,SAL-01
54,SAL,Mercearia,27.0,20.5,SAL-02
55,START,START,15.0,1.0,START


## Recommendation system

## Apriori

### Association Rules

With the order id in place we can now start building a recommendation system based on what items are frequently purchased together. This can be useful for suggesting related items to customers, as well as informing product placement and marketing strategies.

To do this, we use the Apriori algorithm to identify frequent itemsets - that is, sets of items that appear together in a minimum number of orders. From these itemsets, we can generate association rules that tell us which items tend to be purchased together.

The Apriori algorithm is a scalable and efficient way to mine large datasets for frequent itemsets and generate association rules. By using this approach, we can identify patterns and relationships in transactional data that can inform our recommendation system and ultimately help us make data-driven decisions about product recommendations, marketing strategies, and product placement.

In [5]:
%%time
from mlxtend.frequent_patterns import apriori, association_rules

# Filter for orders with more than one item
df = df.groupby('order_id').filter(lambda x: len(x) > 1)

# Pivot data to create binary matrix
basket = pd.pivot_table(df, index='order_id', columns='product_subtype', values='quantity', aggfunc='sum', fill_value=0)

# Convert values to binary
basket[basket > 0] = 1

# Find frequent itemsets
freq_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

freq_itemsets.head(5)

C:\Users\berna\anaconda3\envs\testing\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


CPU times: total: 30.8 s
Wall time: 1min 29s


,support,itemsets
0,0.174519,(Acucar - Adocantes)
1,0.138714,(Aguas Minerais)
2,0.078038,(Aperitivos)
3,0.275317,(Arroz)
4,0.294463,(Azeites)


In the context of the Apriori algorithm, support refers to the frequency with which an itemset appears in the dataset. Specifically, the support of an itemset is defined as the proportion of transactions in the dataset that contain all the items in that itemset.

For example, in the table you provided, the first itemset (`ABOBORA`) has a support value of `0.062928`. This means that about `6.3%` of the transactions in the dataset contain the item `ABOBORA`. Similarly, the second itemset (`ACUCAR GRANULADO SACO PAPEL 1 KG RAR`) has a support value of `0.135336`, which means that about `13.5%` of the transactions in the dataset contain the item `ACUCAR GRANULADO SACO PAPEL 1 KG RAR`.

The support metric is important in the Apriori algorithm because it is used to identify frequent itemsets, which are then used to generate association rules. Specifically, itemsets with a support value above a given threshold (e.g., `0.05` or `0.1`) are considered frequent, and the algorithm generates association rules based on those itemsets.

In [6]:
# Generate association rules
rules_orig = association_rules(freq_itemsets, metric='lift', min_threshold=1)

# Sort rules by lift and support
rules_orig = rules_orig.sort_values(['lift', 'support'], ascending=[False, False])

# Print top 5 rules
print(rules_orig.shape)
rules_orig.head(5)

(200628, 10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4678,"(Sacos de Plastico, Biosog)",(Panos),0.076678,0.061145,0.05115,0.667075,10.909717,0.046461,2.820017,0.983772
4679,(Panos),"(Sacos de Plastico, Biosog)",0.061145,0.076678,0.05115,0.836531,10.909717,0.046461,5.648305,0.967496
4677,"(Panos, Biosog)",(Sacos de Plastico),0.052792,0.102769,0.05115,0.968889,9.427864,0.045724,28.839579,0.943754
4680,(Sacos de Plastico),"(Panos, Biosog)",0.102769,0.052792,0.05115,0.497717,9.427864,0.045724,1.885805,0.996322
4681,(Biosog),"(Panos, Sacos de Plastico)",0.100422,0.054106,0.05115,0.509346,9.413841,0.045716,1.927822,0.993548


### Products likely to be bought in previous orders

In [7]:

#For every order lets compare the product_subtype with antecedents(all that exist) through a metric we choose the antecendent that's more similar with the original order then present for that order the consequents of the most probabily antecedents

# create a dictionary to store the consequents for each antecedent
consequent_dict = {}
for index, row in rules_orig.iterrows():
    antecedent = row['antecedents']
    consequent = row['consequents']
    if antecedent not in consequent_dict:
        consequent_dict[antecedent] = set(consequent)
    else:
        consequent_dict[antecedent].update(consequent)

# create a function to calculate Jaccard similarity
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

# create a dictionary to store the most similar antecedent for each order
similar_antecedent_dict = {}
for order_id in df_new['order_id'].unique():
    order_subtypes = set(df_new.loc[df_new['order_id'] == order_id, 'product_subtype'])
    max_similarity = -np.inf
    for antecedent, consequents in consequent_dict.items():
        antecedent_subtypes = set(antecedent)
        similarity = jaccard_similarity(order_subtypes, antecedent_subtypes)
        if similarity > max_similarity:
            max_similarity = similarity
            max_antecedent = antecedent
    similar_antecedent_dict[order_id] = max_antecedent



This code aims to find the most similar antecedent for each order in the df_new DataFrame, based on the Jaccard similarity between the order's product subtypes and the antecedent's product subtypes in the association rules.

First, a dictionary named **consequent_dict** is created to store the consequents for each antecedent in the rules_orig DataFrame. The keys in the dictionary are the antecedents, and the values are sets of consequents.

Then, a function named **jaccard_similarity** is defined to calculate the Jaccard similarity between two sets.

Next, a dictionary named **similar_antecedent_dict** is created to store the most similar antecedent for each order. The code iterates over each unique order_id in df_new and extracts the product subtypes associated with that order. For each order, the code then iterates over each antecedent in consequent_dict and calculates the Jaccard similarity between the order's product subtypes and the antecedent's product subtypes. The antecedent with the highest similarity is stored in the max_antecedent variable, and the corresponding similarity value is stored in the max_similarity variable. Finally, the most similar antecedent for each order is stored in the similar_antecedent_dict dictionary.

In [8]:
%%time
# create a dictionary to store the consequents for each order
consequent_order_dict = {}
for order_id, antecedent in similar_antecedent_dict.items():
    consequents = consequent_dict[antecedent]
    consequent_order_dict[order_id] = consequents

consequent_order_df = pd.DataFrame(consequent_order_dict.items(), columns=['order_id', 'consequents'])

print(consequent_order_df.shape)
print(df_new.order_id.max())
print("CHECK!")
consequent_order_df.head(10)

(23332, 2)
23332
CHECK!
CPU times: total: 31.2 ms
Wall time: 85.7 ms


,order_id,consequents
0,1,"{Iogurtes, Horticolas Frescos, Frutas Frescas,..."
1,2,"{Iogurtes, Frutas Frescas, Horticolas Frescos,..."
2,3,"{Batata, Horticolas Frescos, Frutas Frescas, P..."
3,4,"{Horticolas Frescos, Frutas Frescas, Horticola..."
4,5,"{Iogurtes, Horticolas Frescos, Frutas Frescas,..."
5,6,"{Queijos, Horticolas Frescos, Frutas Frescas, ..."
6,7,"{Iogurtes, Horticolas Frescos, Frutas Frescas,..."
7,8,"{Batata, Arroz}"
8,9,{Azeites}
9,10,"{Leite, Batata}"


The code generates a dictionary consequent_order_dict which stores the best consequent products for each order, based on the antecedent with the highest Jaccard similarity to the order. Then it converts this dictionary to a DataFrame consequent_order_df with two columns, order_id and consequents, where each row represents an order and its corresponding best consequent products. The shape attribute of the DataFrame gives the number of rows (i.e., the number of orders) and columns. The df_new.order_id.max() gives the highest order ID in the original df_new DataFrame to ensure that the new DataFrame has the correct number of rows. Finally, the head() method displays the first five rows of the DataFrame.

In [9]:
df_new[df_new['order_id']==2]

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id,alley,cellule,alleycell
1,1001096,6001131,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2,AMB-01,02,AMB-01-02
8493,1001096,6001131,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.0,Não,AMB,Taras,Taras (Outras),2,AMB-13,01,AMB-13-01
8494,1001096,6001131,2023-01-03,123000,TARA CAIXA PLAST NOVA F/L,UN,0.0,Não,AMB,Taras,Taras (Outras),2,AMB-13,01,AMB-13-01
64054,1001096,6001131,2023-01-03,574032,FIAMBRE BARRA VACUO ±3KG FDS CAMPOFRIO,KG,6.6,Não,REF,Carne de Porco,Enchidos-Charcutaria Porco,2,REF-03,14,REF-03-14
70035,1001096,6001131,2023-01-03,123871,ACUCAR GRANULADO SACO PAPEL 1 KG RAR,KG,10.0,Não,AMB,Mercearia,Acucar - Adocantes,2,AMB-10,01,AMB-10-01
206624,1001096,6001131,2023-01-03,561661,FRUTOS SILVESTRES CONGELADOS 1KG DIRA,KG,5.0,Não,CON,Frutas,Frutas Congeladas,2,CON-05,06,CON-05-06
208956,1001096,6001131,2023-01-03,627400,PRESUNTO FATIA.PROBAR FOOD SERVICE 500GR,KG,3.0,Não,REF,Carne de Porco,Enchidos-Charcutaria Porco,2,REF-03,14,REF-03-14
223616,1001096,6001131,2023-01-03,571239,CALDO GALINHA PO CALNORT 1 KG,KG,10.0,Não,AMB,Mercearia,Condimentos-Especiarias Importadas,2,AMB-10,13,AMB-10-13
235392,1001096,6001131,2023-01-03,638894,MANTEIGA C/SAL 8G MIMOSA,KG,2.0,Não,REF,Mercearia,Manteigas,2,REF-08,61,REF-08-61
270061,1001096,6001131,2023-01-03,633183,CHOCOLATE CULINARIA 43% RBRAVO 200G CX10,UN,10.0,Não,AMB,Mercearia,Chocolates,2,AMB-10,11,AMB-10-11


- Metrics:
    - antecedents:
        - the antecedent itemset of the association rule
    - consequents: 
        - the consequent itemset of the association rule
    - antecedent support: 
        - the support of the antecedent itemset, i.e., the proportion of transactions that contain all the items in the antecedent
    - consequent support: 
        - the support of the consequent itemset, i.e., the proportion of transactions that contain all the items in the consequent
    - support:
        - the support of the rule, i.e., the proportion of transactions that contain both the antecedent and the consequent
    - confidence: 
        - the confidence of the rule, i.e., the proportion of transactions that contain the consequent given that they also contain the antecedent
    - lift: 
        - the lift of the rule, which measures how much more often the antecedent and consequent co-occur in the dataset than we would expect if they were independent. A lift greater than 1 indicates a positive correlation between the antecedent and consequent, while a lift less than 1 indicates a negative correlation, and a lift equal to 1 indicates independence.
    - leverage: 
        - the leverage of the rule, which measures the difference between the observed frequency of the antecedent and consequent co-occurring and the frequency that would be expected if they were independent. A leverage of 0 indicates independence, while a positive leverage indicates a positive correlation, and a negative leverage indicates a negative correlation.
    - conviction: 
        - the conviction of the rule, which measures how much the antecedent and consequent are dependent on each other. A conviction value greater than 1 indicates that the antecedent and consequent are positively dependent, while a conviction value less than 1 indicates that they are negatively dependent, and a conviction value equal to 1 indicates independence.
    - zhang's metric: 
        - a metric that combines lift and conviction to give an overall measure of the interestingness of the rule. A value greater than 0 indicates that the rule is interesting.
        
The most important metrics for evaluating association rules are usually **support, confidence, and lift**. Support and confidence are used to filter out rules that are not frequent or not strong enough, while lift is used to identify rules that represent interesting correlations between items. In general, a high support value indicates that the rule is frequent, a high confidence value indicates that the rule is strong, and a high lift value indicates that the rule is interesting.

In [10]:
#rules_orig = rules_orig[(rules_orig['confidence']>=0.7) & (rules_orig['support']>= 0.05)]

rules_orig

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4678,"(Sacos de Plastico, Biosog)",(Panos),0.076678,0.061145,0.051150,0.667075,10.909717,0.046461,2.820017,0.983772
4679,(Panos),"(Sacos de Plastico, Biosog)",0.061145,0.076678,0.051150,0.836531,10.909717,0.046461,5.648305,0.967496
4677,"(Panos, Biosog)",(Sacos de Plastico),0.052792,0.102769,0.051150,0.968889,9.427864,0.045724,28.839579,0.943754
4680,(Sacos de Plastico),"(Panos, Biosog)",0.102769,0.052792,0.051150,0.497717,9.427864,0.045724,1.885805,0.996322
4681,(Biosog),"(Panos, Sacos de Plastico)",0.100422,0.054106,0.051150,0.509346,9.413841,0.045716,1.927822,0.993548
...,...,...,...,...,...,...,...,...,...,...
782,(Iogurtes),(Molhos),0.277241,0.218724,0.081417,0.293670,1.342652,0.020778,1.106106,0.353098
47,(Azeites),(Aguas Minerais),0.294463,0.138714,0.053496,0.181673,1.309695,0.012650,1.052496,0.335154
46,(Aguas Minerais),(Azeites),0.138714,0.294463,0.053496,0.385656,1.309695,0.012650,1.148441,0.274547
53,(Horticolas Congelados),(Aguas Minerais),0.354669,0.138714,0.054857,0.154671,1.115030,0.005659,1.018876,0.159861


Explicar

## Q-Learn Hugo e Bernardo

In [11]:
df_new[df_new['order_id']==4822]

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id,alley,cellule,alleycell
6813,1032515,6158810,2023-02-06,620389,IOG AROMA COCO 4X120G REDUC ACU MIMOSA,UN,3.0,Não,REF,Mercearia,Iogurtes,4822,REF-08,59,REF-08-59
138157,1032515,6158810,2023-02-06,560281,BOLACHA MARIA D.I. 25G(CX 144 UN) GULLON,UN,288.0,Não,AMB,Mercearia,Bolachas,4822,AMB-10,06,AMB-10-06
166048,1032515,6158810,2023-02-06,620392,IOG AROMA TFRUTTI 4X120G REDUC ACU MIMOS,UN,2.0,Não,REF,Mercearia,Iogurtes,4822,REF-08,59,REF-08-59
213690,1032515,6158810,2023-02-06,593454,NECTAR JUVER MULTIFRUTOS 30x200ML,UN,60.0,Não,AMB,Bebidas,Sumos,4822,AMB-01,10,AMB-01-10
214550,1032515,6158810,2023-02-06,593455,NECTAR JUVER MACA 30x200ML,UN,90.0,Não,AMB,Bebidas,Sumos,4822,AMB-01,10,AMB-01-10
221083,1032515,6158810,2023-02-06,557121,MACA GOLDEN 70/75,KG,1.0,Não,FRL,Frutas,Frutas Frescas,4822,FRL-01,09,FRL-01-09
260593,1032515,6158810,2023-02-06,639316,NECTAR JUVER PESSEGO E MACA 200ML,UN,90.0,Não,AMB,Bebidas,Sumos,4822,AMB-01,10,AMB-01-10
260886,1032515,6158810,2023-02-06,637657,NECTAR JUVER MACA/ANANAS 30x200ML,UN,30.0,Não,AMB,Bebidas,Sumos,4822,AMB-01,10,AMB-01-10
272862,1032515,6158810,2023-02-06,200584,MELAO VERDE,KG,2.0,Não,FRL,Frutas,Frutas Frescas,4822,FRL-01,09,FRL-01-09
282322,1032515,6158810,2023-02-06,626128,INFUSAO CAMOMILA HOT CX25 SAQ,KI,1.0,Não,AMB,Mercearia,Chas,4822,AMB-10,10,AMB-10-10


In [12]:
rules_final = (rules_orig['antecedents'].apply(lambda x: list(x)) + rules_orig['consequents'].apply(lambda x: list(x))).reset_index()
rules_final.rename(columns = {'index':'order_id',0:'items'}, inplace = True)
rules_final.set_index('order_id', drop=False, inplace=True, verify_integrity=True)
rules_final.rename_axis("",inplace=True)
rules_final

,order_id,items
,,
4678,4678,"[Sacos de Plastico, Biosog, Panos]"
4679,4679,"[Panos, Sacos de Plastico, Biosog]"
4677,4677,"[Panos, Biosog, Sacos de Plastico]"
4680,4680,"[Sacos de Plastico, Panos, Biosog]"
4681,4681,"[Biosog, Panos, Sacos de Plastico]"
...,...,...
782,782,"[Iogurtes, Molhos]"
47,47,"[Azeites, Aguas Minerais]"
46,46,"[Aguas Minerais, Azeites]"


In [13]:
# explode the items column
rules_final = rules_final.explode('items')

# merge the order_id column with the exploded dataframe
rules_final = pd.merge(rules_final, rules_final[['order_id']], left_index=True, right_index=True)

# rename the order_id column to item_order_id
rules_final = rules_final.rename(columns={'order_id_x': 'order_id', 'order_id_y': 'item_order_id'})

rules_final = rules_final[["order_id", "items"]]

rules_final = rules_final.rename(columns = {"items": "product_subtype"})

# display the resulting dataframe
rules_final

,order_id,product_subtype
,,
0,0,Acucar - Adocantes
0,0,Acucar - Adocantes
0,0,Arroz
0,0,Arroz
1,1,Arroz
...,...,...
200627,200627,Horticolas Congelados
200627,200627,Horticolas Congelados
200627,200627,Horticolas Congelados


In [14]:
test_new = df_new.copy()

In [15]:
# create a dictionary with the mapping between product_subtype and product_type
product_type_map = dict(zip(test_new['product_subtype'], test_new['product_type']))


In [16]:
# add a new column to rules_final with the product_type information
rules_final['product_type'] = rules_final['product_subtype'].map(product_type_map)


In [17]:
# Create a dictionary mapping product_type to warehouse_zone, x, y, and alley
prod_type_dict = s.set_index('product_type')[['warehouse_zone', 'x', 'y', 'alley']].to_dict()
prod_type_dict

{'warehouse_zone': {'Bebidas': 'REF',
  'Carne de Bovino': 'REF',
  'Carne de Porco': 'REF',
  'Criacao': 'REF',
  'Equipamentos p/cozinhas': 'REF',
  'Frutas': 'REF',
  'Legumes': 'SAL',
  'Material Informatica': 'NAL',
  'Mercadorias Alimentares': 'AMB',
  'Mercearia': 'SAL',
  'Nao Alimentares': 'NAL',
  'Peixe': 'REF',
  'Taras': 'NAL',
  'Carne de Ovino': 'CON',
  'Material Escritorio': 'NAL',
  'Refeições Cook & Chill (Socigeste)': 'REF',
  'START': 'START'},
 'x': {'Bebidas': 2.5,
  'Carne de Bovino': 2.5,
  'Carne de Porco': 2.5,
  'Criacao': 2.5,
  'Equipamentos p/cozinhas': 2.5,
  'Frutas': 5.5,
  'Legumes': 27.0,
  'Material Informatica': 12.0,
  'Mercadorias Alimentares': 16.0,
  'Mercearia': 27.0,
  'Nao Alimentares': 12.0,
  'Peixe': 5.5,
  'Taras': 12.0,
  'Carne de Ovino': 3.5,
  'Material Escritorio': 12.0,
  'Refeições Cook & Chill (Socigeste)': 5.5,
  'START': 15.0},
 'y': {'Bebidas': 40.0,
  'Carne de Bovino': 38.0,
  'Carne de Porco': 36.0,
  'Criacao': 34.0,
  'Eq

In [18]:

# Use the map method to create new columns in rules_final
rules_final['warehouse_zone'] = rules_final['product_type'].map(prod_type_dict['warehouse_zone'])
rules_final['x'] = rules_final['product_type'].map(prod_type_dict['x'])
rules_final['y'] = rules_final['product_type'].map(prod_type_dict['y'])
rules_final['alley'] = rules_final['product_type'].map(prod_type_dict['alley'])
rules_final

,order_id,product_subtype,product_type,warehouse_zone,x,y,alley
,,,,,,,
0,0,Acucar - Adocantes,Mercearia,SAL,27.0,20.5,SAL-02
0,0,Acucar - Adocantes,Mercearia,SAL,27.0,20.5,SAL-02
0,0,Arroz,Mercearia,SAL,27.0,20.5,SAL-02
0,0,Arroz,Mercearia,SAL,27.0,20.5,SAL-02
1,1,Arroz,Mercearia,SAL,27.0,20.5,SAL-02
...,...,...,...,...,...,...,...
200627,200627,Horticolas Congelados,Legumes,SAL,27.0,25.5,SAL-01
200627,200627,Horticolas Congelados,Legumes,SAL,27.0,25.5,SAL-01
200627,200627,Horticolas Congelados,Legumes,SAL,27.0,25.5,SAL-01


In [19]:
print(rules_final.alley.unique())


['SAL-02' 'SAL-01' 'REF-04' 'REF-03' 'REF-06' 'REF-09' 'REF-01' 'REF-02'
 'NAL-06']


In [20]:
wasdis = rules_final.groupby('order_id')['alley'].nunique()
wasdis.sort_values(ascending=False)

order_id
158262    6
158290    6
158268    6
158269    6
158270    6
         ..
8643      1
8642      1
8641      1
8640      1
0         1
Name: alley, Length: 200628, dtype: int64

In [21]:
test = rules_final[rules_final['order_id'] == 158252]

test

,order_id,product_subtype,product_type,warehouse_zone,x,y,alley
,,,,,,,
158252,158252,Leite,Mercearia,SAL,27.0,20.5,SAL-02
158252,158252,Leite,Mercearia,SAL,27.0,20.5,SAL-02
158252,158252,Leite,Mercearia,SAL,27.0,20.5,SAL-02
158252,158252,Leite,Mercearia,SAL,27.0,20.5,SAL-02
158252,158252,Leite,Mercearia,SAL,27.0,20.5,SAL-02
158252,158252,Leite,Mercearia,SAL,27.0,20.5,SAL-02
158252,158252,Peixe Congelado,Peixe,REF,5.5,34.0,REF-09
158252,158252,Peixe Congelado,Peixe,REF,5.5,34.0,REF-09
158252,158252,Peixe Congelado,Peixe,REF,5.5,34.0,REF-09


In [22]:
import numpy as np
import pandas as pd
import time
import math

start_time = time.time()

# Function for calculating distance between zones
def calculate_distance(zone1, zone2):
    x1, y1 = float(s[s["alley"] == zone1]["x"].iloc[0]), float(s[s["alley"] == zone1]["y"].iloc[0])
    x2, y2 = float(s[s["alley"] == zone2]["x"].iloc[0]), float(s[s["alley"] == zone2]["y"].iloc[0])
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

def get_shortest_distance_zone(zone_list):
    distances = []
    for zone in zone_list:
        distances.append(calculate_distance("START", zone))
    return zone_list[np.argmin(distances)]

# Parameters for Q-learning
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate
n_episodes = 10 #1000  # Number of episodes

# Get the unique order ids and zones
order_ids = test['order_id'].unique()
zones = test['alley'].unique()
min_order_id = np.min(order_ids)

# Initialize the Q-table
num_zones = len(order_ids)
q_table = np.zeros((len(order_ids), len(zones), len(zones)))

for episode in range(n_episodes):
    for order_id in order_ids:
        order = test[test['order_id'] == order_id]
        zone_list = order['alley'].tolist()

        for i in range(len(zone_list) - 1):
            current_zone = zone_list[i]
            next_zone = zone_list[i + 1]

            current_zone_idx = np.where(zones == current_zone)[0][0]
            next_zone_idx = np.where(zones == next_zone)[0][0]

            # Choose action (next zone) using epsilon-greedy strategy
            if np.random.random() < epsilon:
                action_idx = np.random.randint(num_zones)
            else:
                action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :])

            # Calculate the reward based on the distance to the chosen zone
            reward = -calculate_distance(current_zone, zones[min(action_idx, len(zones)-1)])

            # Update Q-table
            q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)] = q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)] + alpha * (reward + gamma * np.max(q_table[order_id - min_order_id, min(action_idx, len(zones)-1), :]) - q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)])

# Print the optimal picking order
for order_id in order_ids:
    order = test[test['order_id'] == order_id]
    zone_list = order['alley'].tolist()

    first_zone = get_shortest_distance_zone(zone_list)
    optimal_order = [first_zone]

    for i in range(len(zone_list) - 1):
        current_zone = zone_list[i + 1]
        current_zone_idx = np.where(zones == current_zone)[0][0]

        # Choose the best action based on the Q-table
        action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :])

        if zones[action_idx] not in optimal_order:
            optimal_order.append(zones[action_idx])

    print(f"Optimal picking order for order {order_id}: {optimal_order}")

end_time = time.time()

print("Running time:", end_time - start_time, "seconds")

Optimal picking order for order 158252: ['SAL-02', 'REF-09', 'REF-03', 'REF-04', 'SAL-01', 'REF-06']
Running time: 0.2973299026489258 seconds
